In [3]:
!pip3 install torchvision


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Set device (GPU/CPU)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom dataset class

In [12]:
class SkinCancerDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # self.data_frame = self.data_frame.loc[self.data_frame['image_type'] == 'dermoscopic']
        img_name = os.path.join(self.img_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name+".jpg").convert('RGB')  # Open image

        self.data_frame['diagnosis'] = LabelEncoder().fit_transform(self.data_frame['diagnosis'])

        label = int(self.data_frame.iloc[idx, 10])

        if self.transform:
            image = self.transform(image)

        return image, label

# Image transformations (data augmentation + normalization)

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to MobileNetV3 input size
    transforms.ToTensor(),          # Convert image to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize to match pre-trained model input
])

# Load CSV and dataset

In [14]:
train_csv = '../data/metadata/train_metadata.csv'
test_csv = '../data/metadata/test_metadata.csv'
image_dir_train = '../data/train_image'
image_dir_test = '../data/test_image'

train_dataset = SkinCancerDataset(csv_file=train_csv, img_dir=image_dir_train, transform=transform)
test_dataset = SkinCancerDataset(csv_file=test_csv, img_dir=image_dir_test, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Load a pre-trained CNN model (ResNet18)

In [15]:
model = models.mobilenet_v3_large(pretrained=True)

# Replace the last fully connected layer for binary classification
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 9)  # 2 output classes: benign and malignant

model = model.to(device)


D:\Workspace\AML\experiment\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Workspace\AML\experiment\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Loss and optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/10], Loss: 1.4565


# Testing loop

In [35]:
model.eval()  # Set model to evaluation mode
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is depreca

Test Accuracy: 46.27%


# Save the trained model

In [ ]:
torch.save(model.state_dict(), 'skin_disease_detection_cnn.pth')